# Proposito
En este notebook estan los pasos a tomar para la etapa de **publicacion** del proceso de **Data Wrangling**. Este proceso recibe como entrada las conclusiones obtenidas en la etapa anterior: **validacion**.

Para este ejemplo, recibiremos como insumo el archivo CSV generado en el paso de `limpieza` llamado `cleaned_data.csv`.

Finalmente, no olvidemos la necesidad del negocio. Necesitamos dar respuesta a la pregunta ¿Que grupo etario trae mas mal riesgo al banco?

# Objetivos
Al finalizar este notebook deberemos de haber:

✅ Respuesto a la pregunta del negocio.

✅ Guardado los datos con los cuales contestamos a la pregunta del negocio.

In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
schema = {
    "Risk": "string",
    "Age": "Int64",
}  # el esquema que hallamos en el proceso anterior

df = pd.read_csv(r"../data/cleaned_data.csv", dtype=schema, usecols=["Risk", "Age"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4811 entries, 0 to 4810
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Risk    4680 non-null   string
 1   Age     4670 non-null   Int64 
dtypes: Int64(1), string(1)
memory usage: 80.0 KB


In [3]:
df: DataFrame = df.dropna()

# Dando respuesta a la pregunta de negocio
Antes de dar respuesta a la pregunta de negocio, arreglemos los problemas encontrados en la etapa anterior.

❌ Eliminemos aquellos datos que no cumplen con las reglas de negocio.

In [ ]:
df: DataFrame = df[
    (df["Age"] >= 22) & (df["Age"] <= 74)  # noqa: PLR2004
]  # eliminar filas que no cumplen con las reglas de negocio.

In [5]:
def get_ages_groups(df: DataFrame) -> DataFrame:
    """
    Agrupa las edades en intervalos de 10 años y agrega una nueva columna con el grupo de edad.

    Args:
        df (pd.DataFrame): DataFrame que contiene una columna llamada "Edad".

    Returns:
        pd.DataFrame: DataFrame con una nueva columna "Grupo_Edad".
    """
    bins = list(range(df["Age"].min(), df["Age"].max() + 1, 10))
    labels = [f"{i}-{i + 9}" for i in bins[:-1]]  # Etiquetas

    df["age_group"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)

    return df

In [6]:
df: DataFrame = get_ages_groups(df)

df.groupby("age_group")["Risk"].value_counts(normalize=True).unstack()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13640\2363714859.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("age_group")["Risk"].value_counts(normalize=True).unstack()


Risk,bad,good
age_group,,
22-31,0.363636,0.636364
32-41,0.237668,0.762332
42-51,0.241477,0.758523
52-61,0.346535,0.653465
62-71,0.221519,0.778481


La respuesta a la pregunta dada es que el grupo etario de los 22 a los 31 son quienes traen mayores malos riesgos al banco, con aproximadamente un 36% de mal riesgo.

In [7]:
df.to_csv(r"../data/final_data.csv", index=False)